<a href="https://colab.research.google.com/github/StevenHSKim/tensorflow_study/blob/main/pj7_%EC%A0%84%EC%9D%B4%ED%95%99%EC%8A%B5%EC%93%B0%EB%A9%B4_5%EB%B6%84_%ED%95%99%EC%8A%B5%EC%97%90_%EC%A0%95%ED%99%95%EB%8F%84_97_%EB%82%98%EC%98%B4_%E3%85%85%E3%84%B1_(Transfer_learning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#전이학습쓰면 5분 학습에 정확도 97% 나옴 ㅅㄱ (Transfer learning)

오늘은 전이학습이란걸 알아보도록 합시다.

Visual Recognition 문제는 수많은 사람들이 이미 도전했던 문제기 때문에  

결과가 매우 좋았던 Conv + Pooling 모델들이 이미 존재합니다. 그 모델을 가져와서 개/고양이 분류문제를 풀어보는겁니다.



1. 구글 Inception V3모델과 weight파일 로드해서 합치기

2. 학습금지 레이어 설정

3. Inception 모델에서 중간에 원하는 레이어만 잘라오기

4. 내 모델 만들고 (특히 출력층 레이어) Inception과 연결

5. 학습 ㄱㄱ

이렇게만 하셔도 우리가 풀던 개/고양이 인식 문제에선 epoch 2번에 정확도 95% 이상 달성가능합니다.

이런 모델 구글이 쓰라고 공개한건데 안쓸이유가 전혀없겠죠?

그래서 이게 요즘 핫한 AI 개발방식이며 많은 대기업 AI 연구원들이 전이학습으로 성과를 날로먹고 있습니다.

## 개/고양이 분류 프로젝트 가져오기

In [4]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

In [7]:
!kaggle competitions download -c dogs-vs-cats

 99% 806M/812M [00:10<00:00, 77.7MB/s]
100% 812M/812M [00:10<00:00, 82.3MB/s]


In [9]:
!unzip -q dogs-vs-cats.zip -d .

In [10]:
!unzip -q train.zip -d .

데이터셋 만드는 부분까지만 일단 실행

In [32]:
import os
import tensorflow as tf
import shutil

os.mkdir('/content/dataset/')
os.mkdir('/content/dataset/cat')
os.mkdir('/content/dataset/dog')

for i in os.listdir('/content/train/'):
  if 'cat' in i:
    shutil.copyfile( '/content/train/' + i , '/content/dataset/cat/' + i )
  if 'dog' in i:
    shutil.copyfile( '/content/train/' + i , '/content/dataset/dog/' + i )

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/dataset/',
  image_size=(150,150),     # 이후에 150x150 사용할 것이기 때문에 이렇게 전처리!
  batch_size=64,
  subset='training',
  validation_split=0.2,
  seed=1234
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/dataset/',
  image_size=(150,150),     # 이후에 150x150 사용할 것이기 때문에 이렇게 전처리!
  batch_size=64,
  subset='validation',
  validation_split=0.2,
  seed=1234
)

print(train_ds)

def 전처리함수(i, 정답):
  i = tf.cast( i/255.0, tf.float32 )
  return i, 정답

train_ds = train_ds.map(전처리함수)
val_ds = val_ds.map(전처리함수)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


#전이 학습에는 (1)모델과 (2)학습완료된 weight가 필요함. 다운 받기
##1. Inception_v3.h5 파일 다운 받기: 이 파일은 weight 파일임

In [33]:
import requests

url = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
r = requests.get(url, allow_redirects=True)

open('inception_v3.h5', 'wb').write(r.content)

87910968

##2. Inception_v3 모델 가져오기. 이때 주의할 점: **"항상 불러와서 사용할 땐 input_shape 생각하기!!!"**

In [34]:
from keras.applications import InceptionV3

inception_model = InceptionV3(input_shape=(150,150,3), # 원래는 299x299x3 사용하지만, 나는 변경
                              include_top=False,       # top layer: 마지막 출력층 (마지막 Dense 레이어)
                              weights=None)            # 위에서 h5 파일 받았기 때문에 여기선 weight 가져오지 않음

##3. weight 연결해주기 (weight 파일 + InceptionV3 합치기)

In [35]:
inception_model.load_weights('inception_v3.h5')

# 모양 확인
inception_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d_188 (Conv2D)         (None, 74, 74, 32)           864       ['input_5[0][0]']             
                                                                                                  
 batch_normalization_188 (B  (None, 74, 74, 32)           96        ['conv2d_188[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_188 (Activation  (None, 74, 74, 32)           0         ['batch_normalizati

*   원래 layer라면 맨 아래 (Dense) output layer가 있었겠지만, include_top=None 이므로 위엔 없는 상황
*   현재 summary에 보이는, 위 레이어들은 Conv+Pooling으로 이미지를 해체/분석해줌
*   우리의 상황(개/고양이 분류)에 맞추어 마지막 output layer 작성해야함

### 이때 주의할 점!!
*   **절대 가져온 레이어는 학습시키면(weight 업데이트 하면) 안됨!**
*   **마지막 우리가 만든 output layer만 학습시켜야 함**


##4. 학습 금지 레이어 설정하기

In [36]:
# inception_model.layers은 각 layer을 모두 한 줄씩 보여줌
# 일단 모두 trainable 잠가두기
for i in inception_model.layers:
  i.trainable = False

##5. 원하는 레이어만 뽑기

In [37]:
# 위 summary를 보면, 덩어리가 mixed로 묶여 있는 것을 볼 수 있음
# 나는 현재 mixed7 layer 까지만 사용하고 싶을 때,

# 마지막 레이어로 mixed7을 가져온다
마지막레이어 = inception_model.get_layer('mixed7')

print(마지막레이어) # 위로 가서 mixed7을 보면 Concatenate을 해주는 layer임을 알 수 있다.
print(마지막레이어.output)
print(마지막레이어.output_shape)

KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 768), dtype=tf.float32, name=None), name='mixed7/concat:0', description="created by layer 'mixed7'")
(None, 7, 7, 768)


##6. 내 레이어에 연결

In [40]:
import tensorflow as tf

# 내 레이어
layer1 = tf.keras.layers.Flatten()(마지막레이어.output) # 앞서 결정한 마지막 레이어를 내가 만든 레이어에 연결
layer2 = tf.keras.layers.Dense(1024, activation='relu')(layer1)
drop1 = tf.keras.layers.Dropout(0.2)(layer2)
layer3 = tf.keras.layers.Dense(1, activation='sigmoid')(drop1)

# 가져온 Inception model의 레이어 + 내 레이어
model = tf.keras.Model(inception_model.input, layer3) # 처음과 끝 (처음: Inception layer의 처음, 끝: 내 레이어의 끝)

# !!내 레이어는 학습 해야함 주의!!
# 딱 내 레이어에만 해당하는 loss 사용하고, opt는 아무거나
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # loss로 binarycrossentropy!!: 개vs고양이 니깐!

# 모델 학습
model.fit(train_ds, validation_data=val_ds, epochs=2)

Epoch 1/2


KeyboardInterrupt: 

너무 오래걸려서 일단 취소
우리가 쌩으로 했을 땐 acc:85% 정도가 한계였는데, 이건 epoch=2밖에 없는데도 97% 정확도

## (참고) Finetuning 하는 법

In [ ]:
# 일단은 위에서 처럼 모두 False로 막아뒀지만,
for i in inception_model.layers:
  i.trainable = False

# 예를들어 Inception 모델을 mixed6부터 트레이닝을 조금 하고 싶다면:
unfreeze = False
for i in inception_model.layers:
  if i.name == 'mixed6':
    unfreeze = True
  if unfreeze == True:
    i.trainable = True

# 위 코드로 원하는 layer을 unfreeze해준 후,
# 아래처럼 optimizer를 바꿔주면 된다. optimizer를 learning rate를 아주 약간만 줘서, 미세하게 튜닝(fine tuning)하는 것!
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.00001), metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=2)

# (참고) 기존 compile 코드: model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])